In [8]:
import pandas as pd
import os
import time

In [44]:
net = pd.read_csv(os.path.join("clean_data", "net.csv"))
feat = pd.read_csv(os.path.join("mapped_feat", "weighted_feat2.csv"))

In [45]:
net.head()

,Unnamed: 0,pos,value,chr_num,reads,strand
0,1,156,0.481988,1,1.0,+
1,2,5793,0.481988,1,1.0,+
2,3,5871,0.481988,1,1.0,+
3,4,5883,0.481988,1,1.0,+
4,5,5889,0.481988,1,1.0,+


In [46]:
feat.head()

,Unnamed: 0.1,Unnamed: 0,chr_num,left,right,strand,class,reads
0,0,1,1,7276,9260,-,ORFs,98.777148
1,1,2,1,31152,32984,+,ORFs,1333.419131
2,2,3,1,33360,34896,+,ORFs,859.659276
3,3,4,1,35096,36392,+,ORFs,1225.092444
4,4,5,1,36544,37328,+,ORFs,833.895828


In [47]:
feat.sort_values(by = ["strand", "chr_num", "left"]).iloc[4:8]

,Unnamed: 0.1,Unnamed: 0,chr_num,left,right,strand,class,reads
6097,6097,6098,1,9368,9600,+,SUTs,74.0
8938,8938,8939,1,10091,10399,+,CDS,7.0
7272,7272,7273,1,11270,11785,+,XUTs,417.0
8941,8941,8942,1,12046,12426,+,ORF,5.5


In [48]:
# net = net.iloc[800:805]
net = net.iloc[:10000]
net["total"] = 0.0
net["feature"] = None

In [49]:
t0 = time.time()
i = 0
for d in net.itertuples():
    if i != d.chr_num:
        i = d.chr_num
        print(f"Now on chromosome: {i}")
        t = time.time() - t0
        print(f"Time is {t//3600} hours {(t%3600) // 60} minutes {(t%3600) % 60} seconds")

    condition1 = (feat["strand"] == d.strand)
    condition2 = (feat["chr_num"] == d.chr_num) 
    condition3 = (feat["left"] < d.pos)
    condition4 = (feat["right"] > d.pos)
    df = feat[condition1 & condition2 & condition3 & condition4]
    if len(df) != 0:
        net.at[d.Index, "total"] = df["reads"].sum()
        net.at[d.Index, "feature"] = df[df["reads"] == max(df["reads"])]["class"].iloc[0]

Now on chromosome: 1
Time is 0.0 hours 0.0 minutes 0.005029439926147461 seconds


In [50]:
net

,Unnamed: 0,pos,value,chr_num,reads,strand,total,feature
0,1,156,0.481988,1,1.0,+,0.0,None
1,2,5793,0.481988,1,1.0,+,0.0,None
2,3,5871,0.481988,1,1.0,+,0.0,None
3,4,5883,0.481988,1,1.0,+,0.0,None
4,5,5889,0.481988,1,1.0,+,0.0,None
...,...,...,...,...,...,...,...,...
9995,9996,62337,2.891927,1,6.0,+,4966.0,ORFs
9996,9997,62338,1.445963,1,3.0,+,4966.0,ORFs
9997,9998,62343,0.481988,1,1.0,+,4966.0,ORFs
9998,9999,62344,0.481988,1,1.0,+,4966.0,ORFs


In [51]:
time.time() - t0

53.55935454368591

In [62]:
df

,Unnamed: 0.1,Unnamed: 0,chr_num,left,right,strand,class,reads
19,19,20,1,61278,62646,+,ORFs,1753.77274
8989,8989,8990,1,61316,62563,+,ORF,1606.11363
8990,8990,8991,1,61316,62563,+,CDS,1606.11363
